In [5]:
import os
from collections import Counter
from pathlib import Path
import pandas as pd

In [25]:
raw_data_path = '../raw_data/'
raw_data_dir_names = ['ethan', 'sai']
raw_data_dirs = [raw_data_path+raw_data_dir_name for raw_data_dir_name in raw_data_dir_names]

In [48]:
counter = Counter()
paths = {}
for dir in raw_data_dirs:
    for file in Path(dir).rglob("*"):
        if file.is_file():
            doc_format = file.suffix.lower() if file.suffix else "no_extension"
            counter[doc_format]+=1
            list_of_paths = paths.get(doc_format,[])
            list_of_paths.append(str(file))
            paths[doc_format] = list_of_paths

In [52]:
documents_summary_df = pd.DataFrame(counter.items(), columns=["Document Format", "Document Count"])
documents_summary_df

,Document Format,Document Count
0,.ris,2
1,.html,43
2,.pdf,253
3,no_extension,2
4,.docx,1


In [63]:
valid_extensions = ['.html','.pdf','.docx']
metadata_extension = ['.ris']
total_documents = documents_summary_df[documents_summary_df["Document Format"].isin(valid_extensions)]["Document Count"].sum()

print(f"Total Document Count: {total_documents}")

Total Document Count: 297
